In [ ]:
!git clone https://github.com/chocolatedesue/vits.git
%cd vits
!git checkout dev
!pip install onnx.txt 
%cd monotonic_align
!python setup.py build_ext --inplace


In [ ]:
import utils
hps = utils.get_hparams_from_file("/root/workspace/vits/logs/all_forth/all_config.json")
mul_ct_path = "/root/workspace/vits/logs/all_fif/G_first.pth"
import torch
device = torch.device('cuda')



In [ ]:
from models import SynthesizerTrn
from text.symbols import symbols
net_g = SynthesizerTrn(
    len(symbols),
    hps.data.filter_length // 2 + 1,
    hps.train.segment_size // hps.data.hop_length,
    n_speakers=hps.data.n_speakers,
    **hps.model).to(device)
_ = net_g.eval()

_ = utils.load_checkpoint(mul_ct_path, net_g, None)

from text import text_to_sequence
import commons


def get_text(text, hps):
    text_norm = text_to_sequence(text, hps.data.text_cleaners)
    if hps.data.add_blank:
        text_norm = commons.intersperse(text_norm, 0)
    text_norm = torch.LongTensor(text_norm)
    return text_norm

In [ ]:
input = "こにちわ、あやせです"
dummy_input = get_text(input,hps)
print (type(dummy_input),dummy_input,sep='\n')

In [ ]:
torch.onnx.export(net_g, dummy_input, "net_g.onnx", verbose=True,export_params=True, opset_version=14)